<a href="https://colab.research.google.com/github/tylerjamesryan/Simulation/blob/master/irtree_simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
if (!require(mirt)) {
  install.packages("mirt")
} else {
  library(mirt)
}

library(Matrix)
library(parallel)

par_bias <- function(true_par, est_par) {
  return(mean(est_par - true_par, na.rm = T))
}

par_SE <- function(true_par, est_par) {
  return(sqrt(mean((est_par - mean(est_par))^2)))
}

par_RMSE <- function(true_par, est_par) {
  return(sqrt((mean((est_par - true_par)^2))))
}

par_fit <- function(pars) {
  pbias <- par_bias(pars[, "true"], pars[,"model"])
  pRMSE <- par_RMSE(pars[, "true"], pars[,"model"])
  pSE <- par_SE(pars[, "true"], pars[,"model"])
  return(c("Bias" = pbias, "RMSE" = pRMSE, "SE" = pSE))
}

dendrify_item <- function(x, tmat) {
  if (min(x) < 1) { # check that minimum item score > 0
    x + (1 - min(x)) # make minimum item score = 1 if need be
  }
  if (is.null(colnames(tmat))) { # assign node col names if none present in tmat
    colnames(tmat) <- paste0("node_", 1:ncol(tmat)) 
  }
  return(tmat[x, ]) # index row of tmat by item response integer/vector x
}

dendrify <- function(x, tmat, long = FALSE) {
  J <- ncol(x) # number of items
  N <- nrow(x) # number of respondents
  K <- ncol(tmat) # number of latent variables
  X <- matrix(nrow = N, ncol = J * K,
              dimnames = list(1:N, 1:(J*K)))
  for (i in 1:J) {
    x_i <- dendrify_item(x[, i], tmat)
    X[, c((i + J * 1:K) - J)] <- x_i
    colnames(X)[c((i + J * 1:K) - J)] <- paste0(colnames(x_i), "_", "i", i)
  }
  if (long) {
    c_names <- rep(colnames(X), each = N)
    c_names <- do.call("rbind", stringr::str_split(c_names, "_i"))
    colnames(c_names) <- c("node", "item")
    id <- rep(1:N, J*K)
    response <- matrix(c(X), N*J*K, 1)
    X <- cbind.data.frame(id, c_names, "response" = response)
  }
  return(X)
}

de_dendrify <- function(data, tmat) {
  M <- nrow(tmat)
  N <- nrow(data)
  JK <- ncol(data)
  K <- ncol(tmat)
  J <- JK / K
  data_new <- matrix(NA, N, J, dimnames = list(1:N, paste0("i", 1:J)))
  tmat_cond <- matrix(as.numeric(rownames(tmat)), nrow = M)
  rownames(tmat_cond) <- apply(tmat, 1, function(m) paste0(m, collapse = ""))
  for (j in 1:J) {
    data_j <- data[, ((J * 0:(K-1)) + j)]
    data_j_collapse <- apply(data_j, 1, function(i) paste0(i, collapse = ""))
    data_new[, j] <- tmat_cond[data_j_collapse, ]
  }
  return(data_new)
}

grm_data_gen <- function(alpha, beta, theta, return_prob = F) {
  n <- nrow(theta)
  j <- nrow(beta)
  m <- ncol(beta)
  k <- ncol(theta)
  aTh <- theta %*% t(alpha)
  aTh_m <- aTh %*% t(as.matrix(Matrix::bdiag(replicate(j, matrix(1, nrow = m, ncol = 1), simplify = F))))
  alpha_m <- alpha %*% as.matrix(Matrix::bdiag(replicate(k, matrix(1, ncol = m), simplify = F)))
  beta_m <- beta %*% do.call(cbind, replicate(k, diag(m), simplify = F))
  B_m <- alpha_m * beta_m
  B_m_n <- matrix(1, ncol = 1, nrow = n) %*% t(matrix(c(t(B_m %*% t(do.call(cbind, replicate(k, diag(m), simplify = F)))))))
  Z <- aTh_m - B_m_n
  brf <- 1 / (1 + exp(-Z))
  brf_m <- matrix(c(t(brf)), ncol = m, byrow = T)
  Pr_m <- cbind(1, brf_m) - cbind(brf_m, 0)
  response <- apply(Pr_m, 1, function(pr) sample(1:(m + 1), 1, replace = F, prob = pr))
  response <- matrix(response, ncol = j, nrow = n, byrow = T)
  if (return_prob) {
    Pr_m <- split.data.frame(Pr_m, rep(1:j, each = n))
    return(list("response" = response, "prob" = Pr_m))
  } else {
    return(response)
  }
}

irtree_data_gen <- function(alpha, beta, theta, T_matrix, return_prob = F) {
  n <- nrow(theta <- as.matrix(theta))
  j <- nrow(as.matrix(beta))
  k <- ncol(as.matrix(theta))
  q <- ncol(T_matrix)
  j_q <- j/q
  m <- nrow(T_matrix)
  T_matrix_rev <- abs(T_matrix - 1) # convert 0s to 1s and 1s to 0s
  
  # multiply alpha * beta and create an n x j*3 matrix of weighted difficulty parameters
  # this is overly complex, need to simplify
  d <- matrix(sqrt(rowSums(alpha * alpha))) %*% matrix(1, 1, ncol(beta)) * beta
  B <- matrix(1, ncol = j, nrow = n) %*% diag(c(d %*% matrix(1, nrow = ncol(beta))))
  
  # multiply alpha * theta and create an n x j*3 matrix of weighted thetas
  aTH <- (theta %*% t(alpha))
  
  # calculate ability - item difficulty discrepancy
  Z <- aTH - B
  
  # estimate response probabilities for a given node
  Pr <- 1 / (1 + exp(-Z))
  
  response <- matrix(nrow = n, ncol = j_q)
  probs <- vector("list", j_q)
  # iterate through items
  for (i in 1:j_q) {
    
    # matrix of possible response options for a given item
    response_options <- matrix(nrow = n, ncol = m)
    
    # selection probabilities for a given item for all nodes
    i_nodes <- (i + j_q*(1:q)) - j_q
    Pr_i <- Pr[, i_nodes]
    
    # iterate through response options
    for (r in 1:m) {
      
      # create an n x 3 matrix for a given response option from the T matrix
      T_matrix_rev_r <- matrix(1, n) %*% T_matrix_rev[r, , drop = F]
      
      # code probabilities according to T matrix and multiply across nodes for overall response probability
      response_options[, r] <- apply(abs(Pr_i - T_matrix_rev_r), 1, prod, na.rm = T)
    }
    probs[[i]] <- response_options
    # draw samples from response proabilities for a given item
    response[, i] <- apply(response_options, 1, function(x) sample(1:m, 1, prob = x))
  }
  
  # create item names
  colnames(response) <- paste0("i", 1:j_q)
  
  dendrified_response <- dendrify(response, T_matrix)
  ret <- list("Likert" = response, "Tree" = dendrified_response)
  if (return_prob) {
    ret <- c(ret, list("Likert_prob" = probs))
  }
  return(ret)
}

irtree_mpp_predict <- function(mod, newdata = NULL, T_matrix = NULL) {
  require(mirt)
  if (is.null(T_matrix)) {
    T_matrix <- matrix(c(0, 0, 1,
                         0, 0, 0,
                         1, NA, NA,
                         0, 1, 0,
                         0, 1, 1),
                       nrow = 5, byrow = T)
  }
  if (!is.null(newdata)) {
    thetas <- do.call("rbind", lapply(1:nrow(newdata), function(i) 
      fscores(mod, response.pattern = newdata[i,], 
              append_response.pattern = F)))
    thetas <- thetas[, -grep("SE_", colnames(thetas)), drop = F]
  } else {
    thetas <- fscores(mod, QMC = T)
  }
  tree_J <- extract.mirt(mod, "nitems")
  K <- ncol(T_matrix)
  M <- nrow(T_matrix)
  J <- tree_J / K
  n <- nrow(as.matrix(thetas))
  tree_item_names <- extract.mirt(mod, "itemnames")
  Pr <- matrix(nrow = n, ncol = tree_J,
               dimnames = list(1:n, tree_item_names))
  for (i in 1:tree_J) {
    item_j <- extract.item(x = mod, item = i)
    Pr[, i] <- expected.item(x = item_j, Theta = thetas)
  }
  
  response <- matrix(nrow = n, ncol = J, dimnames = list(1:n, paste0("i", 1:J)))
  T_matrix <- abs(T_matrix - 1) # convert 0s to 1s and 1s to 0s
  
  
  # recode tree data into likert data
  rs_probs <- vector(mode = "list", length = J)
  for (i in 1:J) {
    pseud_i <- i + rep(J, K) * (0:(K-1))
    response_options <- matrix(nrow = n, ncol = M)
    Pr_i <- Pr[, pseud_i]
    for (r in 1:M) {
      
      # create an n x 3 matrix for a given response option from the T matrix
      T_matrix_r <- matrix(1, nrow(thetas)) %*% T_matrix[r, ]
      
      # code probabilities according to T matrix and multiply across nodes for overall response probability
      response_options[, r] <- apply(abs(Pr_i - T_matrix_r), 1, prod, na.rm = T)
    }
    
    # draw samples from response proabilities for a given item
    response[, i] <- apply(response_options, 1, function(x) sum(x * 1:M))
    rs_probs[[i]] <- response_options
  }
  
  # create item names
  colnames(response) <- names(rs_probs) <- paste0("i", 1:J)
  return(list("rs" = response, "tree" = round(Pr), "rs_prob" = rs_probs, "tree_prob" = Pr))
}

grm_predict <- function(mod, newdata = NULL, theta = NULL) {
  j <- extract.mirt(mod, "nitems")
  
  if (!is.null(newdata)) {
    observed <- newdata
    theta <- do.call("rbind", lapply(1:nrow(newdata), function(i) 
      fscores(mod, response.pattern = newdata[i,], 
              append_response.pattern = F)))
    theta <- theta[, -grep("SE_", colnames(theta)), drop = F]
    
  } else if (is.null(theta)) {
    observed <- extract.mirt(mod, "data")
    theta <- fscores(mod)
  }
  
  items <- lapply(1:j, function(i) extract.item(mod, item = i))
  pred <- sapply(1:j, function(i) expected.item(items[[i]], theta, min = 1))
  colnames(pred) <- colnames(newdata)
  return(pred)
}

rorme <- function(observed, predicted, theta = NULL) {
  if (!all(dim(observed) == dim(predicted))) {
    stop("Observed and Predicted matrices do not have the same dimensions")
  }
  n <- nrow(observed)
  j <- ncol(observed)
  rmse1 <- sqrt(mean((observed - predicted)^2))
  rmse0 <- sqrt(mean((mean(observed) - predicted)^2))
  rse <- sum((observed - predicted)^2) / sum(((mean(observed) - observed))^2)
  rorme <- rmse1 / rmse0
  modelfit <- c("RMSE" = rmse1, "RSE" = rse, "RORME" = rorme)
  
  item_rmse1 <- sqrt(colMeans((observed - predicted)^2))
  item_rmse0 <- sqrt(colMeans((matrix(1, n, 1) %*% colMeans(observed) - predicted)^2))
  item_rorme <- item_rmse1 / item_rmse0
  itemfit <- list("RMSEi" = item_rmse1, "RORMEi" = item_rorme)
  person_rmse1 <- sqrt(rowMeans((observed - predicted)^2))
  person_rmse0 <- sqrt(rowMeans((rowMeans(observed) %*% matrix(1, 1, j) - predicted)^2))
  person_rorme <- person_rmse1 / person_rmse0
  personfit <- list("RMSEp" = person_rmse1, "RORMEp" = person_rorme)
  
  return(list("modelfit" = modelfit, "itemfit" = itemfit, "personfit" = personfit))
}

extract_tree_pars <- function(mod, alpha, beta, theta, phi, iter = NULL, node_names = NULL) {
  j <- nrow(as.matrix(beta))
  n <- nrow(theta)
  K <- ncol(theta)
  j_k <- j/K
  if (class(mod) == "try-error") {
    item_pars <- matrix(NA, nrow = j, ncol = K + 3,
                        dimnames = list(1:j, c(paste0("a", 1:K), "d", "g", "u")))
    item_pars <- as.data.frame(item_pars)
    item <- rep(1:j_k, K)
  } else { 
    mod_coef <- coef(mod, simplify = T)
    item_pars <- as.data.frame(mod_coef$items)
    item <- gsub("\\.\\w\\d{1,}", "", row.names(item_pars))
    item <- as.numeric(gsub("\\D", "", item))
  }
  item_pars$item <- item
  if (is.null(node_names)) {
    node <- gsub("i\\d{1,}\\.", "", row.names(item_pars))
    node <- gsub("\\d{1,}", "", node)
  } else {
    node <- rep(node_names, each = j_k)
  }
  item_pars$node <- node
  #item_pars <- item_pars[order(item_pars$node, item_pars$item), ]
  # alpha parameters
  
  a_true <- rowSums(alpha)
  a_mod  <- rowSums(item_pars[grep("a\\d", names(item_pars))])
  a_pars <- cbind.data.frame("n" = n, "j" = j_k, "iter" = iter,
                             "node" = item_pars$node, 
                             "item" = item_pars$item, 
                             "true" = a_true, 
                             "model" = a_mod)
  
  # beta parameters
  
  b_true <- rowSums(as.matrix(beta))
  b_mod  <- rowSums(item_pars[c("d")]) / -a_mod
  b_pars <- cbind.data.frame("n" = n, "j" = j_k, "iter" = iter,
                             "node" = item_pars$node, 
                             "item" = item_pars$item, 
                             "true" = b_true, 
                             "model" = b_mod)
  
  # latent covariances
  
  phi_true <- phi[lower.tri(phi)]
  phi_mod <- cov2cor(mod_coef$cov)[lower.tri(mod_coef$cov)]
  phi_pars <- cbind.data.frame("n" = n, "j" = j, "iter" = iter,
                               "true" = phi_true,
                               "model" = phi_mod)
  
  if (!is.null(theta)) {
    if (class(mod) == "try-error") {
      EAP <- matrix(NA, n, K)
    } else {
      EAP <- fscores(mod, QMC = T)
    }
    theta_param <- cbind.data.frame("n" = n, "j" = j_k, "iter" = iter,
                                    "node" = rep(unique(node), each = n),
                                    "id" = rep(1:n, K),
                                    "model" = c(EAP),
                                    "true" = c(as.matrix(theta)))
  }
  
  pars_list <- list("alpha" = a_pars, "beta" = b_pars, "phi" = phi_pars, "theta" = theta_param)
  
  return(pars_list)
}

append_sim <- function(sims, sim) {
  if (length(sims) != length(sim)) {
    stop("Doesn't Work Bub...")
  }
  for (l in 1:length(sim)) {
    sims[[l]] <- rbind(sims[[l]], sim[[l]])
  }
  return(sims)
}

rlkj <- function (K, n = 1, eta = 1) {
  # taken from rethinking package (McElreath, 2020)
  stopifnot(is.numeric(K), K >= 2, K == as.integer(K))
  stopifnot(eta > 0)
  f <- function() {
    alpha <- eta + (K - 2)/2
    r12 <- 2 * rbeta(1, alpha, alpha) - 1
    R <- matrix(0, K, K)
    R[1, 1] <- 1
    R[1, 2] <- r12
    R[2, 2] <- sqrt(1 - r12^2)
    if (K > 2) 
      for (m in 2:(K - 1)) {
        alpha <- alpha - 0.5
        y <- rbeta(1, m/2, alpha)
        z <- rnorm(m, 0, 1)
        z <- z/sqrt(crossprod(z)[1])
        R[1:m, m + 1] <- sqrt(y) * z
        R[m + 1, m + 1] <- sqrt(1 - y)
      }
    return(crossprod(R))
  }
  R <- replicate(n, f())
  if (dim(R)[3] == 1) {
    R <- R[, , 1]
  }
  else {
    R <- aperm(R, c(3, 1, 2))
  }
  return(R)
}

get_irtree_data <- function(N, J, tmat, type = "2PL", pars = TRUE) {
  K <- ncol(tmat)
  M <- nrow(tmat)
  alpha <- (diag(K) %x% matrix(1, J))
  if(type == "2PL") {
    alpha <- diag(rlnorm(J*K, .22, .33)) %*% alpha
  } 
  beta <- matrix(rnorm(J*K, 0, 1))
  phi <- rlkj(K = K)
  theta <- replicate(K, rnorm(N, 0, 1)) %*% chol(phi)
  X <- irtree_data_gen(alpha, beta, theta, tmat)
  if (pars) {
    X$pars <- list(alpha = alpha, beta = beta, phi = phi, theta = theta)
  }
  return(X)
}

lmat2mirtmod <- function(lmat, J, COV) {
  require(mirt)
  if (missing(COV)) {
    COV <- diag(ncol(lmat))
    COV <- upper.tri(COV)
  }
  return(mirt.model(abs(lmat) > 0, COV = COV))
}

get_irtree_sim_results <- function() {
  rds_files <- list.files("results/", "\\.rds", full.names = T)
  file_names <- gsub("results\\/\\/|\\.rds", "", rds_files)
  L <- length(rds_files)
  models_list <- vector("list", L)
  pars_list <- vector("list", L)
  for (f in 1:L) {
    file <- readRDS(rds_files[f])
    L_f <- length(file)
    models_list_f <- vector("list", L_f)
    pars_list_f <- vector("list", L_f)
    for (i in 1:L_f) {
      models_list_f[[i]] <- file[[i]]$model
      pars_list_f[i] <- list(file[[i]]$pars)
    }
    models_list[[f]] <- models_list_f
    pars_list[[f]] <- pars_list_f
  }
  names(models_list) <- names(pars_list) <- file_names
  return(list("models" = models_list, "pars" = pars_list))
}



irtree_mpp_predict <- function(mod, newdata = NULL, T_matrix = NULL) {
  require(mirt)
  if (is.null(T_matrix)) {
    T_matrix <- matrix(c(0, 0, 1,
                         0, 0, 0,
                         1, NA, NA,
                         0, 1, 0,
                         0, 1, 1),
                       nrow = 5, byrow = T)
  }
  if (!is.null(newdata)) {
    thetas <- do.call("rbind", lapply(1:nrow(newdata), function(i) 
      fscores(mod, response.pattern = newdata[i,], 
              append_response.pattern = F)))
    thetas <- thetas[, -grep("SE_", colnames(thetas)), drop = F]
  } else {
    thetas <- fscores(mod, QMC = T)
  }
  tree_J <- extract.mirt(mod, "nitems")
  K <- ncol(T_matrix)
  M <- nrow(T_matrix)
  J <- tree_J / K
  n <- nrow(as.matrix(thetas))
  tree_item_names <- extract.mirt(mod, "itemnames")
  Pr <- matrix(nrow = n, ncol = tree_J,
               dimnames = list(1:n, tree_item_names))
  for (i in 1:tree_J) {
    item_j <- extract.item(x = mod, item = i)
    Pr[, i] <- expected.item(x = item_j, Theta = thetas)
  }
  
  response <- matrix(nrow = n, ncol = J, dimnames = list(1:n, paste0("i", 1:J)))
  T_matrix <- abs(T_matrix - 1) # convert 0s to 1s and 1s to 0s
  
  
  # recode tree data into likert data
  rs_probs <- vector(mode = "list", length = J)
  for (i in 1:J) {
    pseud_i <- i + rep(J, K) * (0:(K-1))
    response_options <- matrix(nrow = n, ncol = M)
    Pr_i <- Pr[, pseud_i]
    for (r in 1:M) {
      
      # create an n x 3 matrix for a given response option from the T matrix
      T_matrix_r <- matrix(1, nrow(thetas)) %*% T_matrix[r, ]
      
      # code probabilities according to T matrix and multiply across nodes for overall response probability
      response_options[, r] <- apply(abs(Pr_i - T_matrix_r), 1, prod, na.rm = T)
    }
    
    # draw samples from response proabilities for a given item
    response[, i] <- apply(response_options, 1, function(x) sum(x * 1:M))
    rs_probs[[i]] <- response_options
  }
  
  # create item names
  colnames(response) <- names(rs_probs) <- paste0("i", 1:J)
  return(list("rs" = response, "tree" = round(Pr), "rs_prob" = rs_probs, "tree_prob" = Pr))
}


vech_item_pars <- function(mod, true_pars = NULL) {
  fit_pars <- as.data.frame(coef(mod, printSE = mod@Options$SE, as.data.frame = T))
  par_id <- as.data.frame(stringr::str_split_fixed(row.names(fit_pars), "\\.", n = 2))
  fit_pars <- cbind(par_id, fit_pars)
  names(fit_pars) <- c("id", "par", "est", "SE")
  if (!is.null(true_pars)) {
    fit_pars$true <- vech_pars(tau = true_pars$tau, group_pars = true_pars$group_pars)
  } else {
    return(cbind(fit_pars, "true" = NA))
  }
  
  return(fit_pars)
}

vech_person_pars <- function(mod, true_theta) {
  N <- nrow(as.matrix(true_theta))
  K <- ncol(as.matrix(true_theta))
  QMC <- K > 2
  con <- try(v_fitted_th <- fscores(mod, QMC = QMC, full.scores.SE = T))
  if (class(con)[1] == "try-error") {
    v_fitted_th <- matrix(NA, N*(K*2), 1)
  }
  v_true_theta <- c(true_theta)
  
  person_pars <- data.frame(
    "id" = paste0("p_", rep(1:N, times = K)),
    "par" = paste0("th_", rep(1:K, each = N)),
    "est" = c(v_fitted_th[,1:K]),
    "SE" = c(v_fitted_th[,(K+1):(K*2)]),
    "true" = v_true_theta
  )
  return(person_pars)
}

vech_pars <- function(tau, group_pars) {
  K <- length(grep("\\ba", colnames(tau)))
  M <- length(grep("\\bd", colnames(tau)))
  tau[, (K+1):(K+M)] <- matrix(-1*sqrt(rowSums(tau[,1:K] * tau[,1:K]))) * tau[, (K+1):(K+M)]
  
  return(c(c(t(tau)), c(group_pars$zeta), group_pars$sigma[lower.tri(group_pars$sigma, diag = T)]))
}

get_pars <- function(mod, true_pars, theta) {
  if (missing(true_pars)) {
    person_pars <- vech_person_pars(mod, theta)
    con <- check_converged(mod)
    df_pars <- cbind.data.frame(person_pars, "converged" = con)
    return(df_pars)
  }
  item_pars <- vech_item_pars(mod, true_pars)
  person_pars <- vech_person_pars(mod, theta)
  df_pars <- rbind(item_pars, person_pars)
  
  con <- check_converged(mod)
  df_pars$converged <- con
  return(df_pars)
}

check_converged <- function(mod) {
  mod@OptimInfo$converged
}




make_tau <- function(alpha, beta, theta, g = 0, u = 0) {
  alpha <- as.matrix(alpha)
  colnames(alpha) <- paste0("a", 1:ncol(alpha))
  beta <- matrix(-sqrt(rowSums(alpha * alpha)) * beta)
  colnames(beta) <- paste0("d", 1:ncol(beta))
  tau <- cbind(alpha, beta, "g" = g, "u" = u)
  return(tau)
}

gen_data_f <- function(n, j, b, tmat) {
  K <- ncol(tmat)
  
  phi <- diag(K)
  omega <- diag(K)
  sigma <- omega %*% phi %*% omega
  zeta <- matrix(rep(0, K))
  group_pars <- list("phi" = phi, "omega" = omega,
                     "sigma" = sigma, "zeta" = zeta)
  
  beta <- matrix(c(rnorm(j, b, 1), rnorm(j*(K-1), 0, 1)))
  alpha <- as.matrix(bdiag(replicate(K, rep(1, j), simplify = F)))
  #alpha <- as.matrix(bdiag(replicate(K, rlnorm(j, 0, .25), simplify = F)))
  theta <- replicate(K, rnorm(n, 0, 1)) %*% chol(sigma)
  tau <- make_tau(alpha, beta, theta)
  
  Y <- irtree_data_gen(alpha, beta, theta, tmat)
  list("Y" = Y, "tau" = tau, "group_pars" = group_pars, "theta" = theta)
}

mod_f <- function(Y, ...) {
  model <- lmat2mirtmod(Y$tau[,grep("\\ba", colnames(Y$tau))], COV = F)
  try(fit <- mirt(Y$Y$Tree, model = model, ...))
  return(fit)
}

fit_fun <- function(n, j, b, tmat, i) {
  i_check <- TRUE
  while (i_check) {
    Y <- gen_data_f(n = n, j = j, b = b, tmat = tmat)
    i_check <- any(!apply(Y$Y$Tree, 2, function(i) length(unique(na.omit(i)))) > 1)
  }
  fit <- mod_f(Y, itemtype = "Rasch", method = "MHRM", technical = list(NCYCLES = 1e5), SE = T)
  v_pars <- get_pars(mod = fit, true_pars = Y[2:3], theta = Y$theta)
  v_pars <- cbind("n" = n, 
                  "j" = j,
                  "iter" = i, 
                  "b1" = b,
                  "tmat" = ncol(tmat)-1,
                  v_pars)
  v_pars
}

tmat1 <- matrix(c(0,NA,
                  1, 0,
                  1, 1), 
                nrow = 3, byrow = T,
                dimnames = list(1:3, c("n1", "n2")))

tmat2 <- matrix(c(0,NA,NA,
                  1, 0,NA,
                  1, 1, 0,
                  1, 1, 1), 
                nrow = 4, byrow = T,
                dimnames = list(1:4, c("n1", "n2", "n3")))

tmat3 <- matrix(c(0,NA,NA,NA,
                  1, 0,NA,NA,
                  1, 1, 0,NA,
                  1, 1, 1, 0,
                  1, 1, 1, 1), 
                nrow = 5, byrow = T,
                dimnames = list(1:5, c("n1", "n2", "n3", "n4")))





Loading required package: mirt

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mirt’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘permute’, ‘GPArotation’, ‘vegan’, ‘Deriv’, ‘dcurver’, ‘RcppArmadillo’




In [ ]:
library(mirt)
rdf <- data.frame()

In [ ]:
bs <- c(-2, 0, 2)
tmats <- list(tmat1, tmat2, tmat3)
iters <- 6
Ns <- c(500, 1000)
Js <- c(10, 20)
start_iter <- 1
end_iter <- 10
for (i in start_iter:end_iter) {
  for (b in bs) {
    for (n in Ns) {
      for (j in Js) {
        for (tmat in tmats) {

          t1 <- Sys.time()
          fit <- fit_fun(n, j, b, tmat, i)
          fit$simID <- paste0("n_", n, "j_", j, "b_", b, "tmat_", fit$tmat[1], "iter_", i)
          rdf <- rbind(rdf, fit)
          t2 <- Sys.time()
          cat("n =", n, "; j =", j, "; b =", b, "; iter =", i, "; time =", (t2 - t1)[[1]], "\n")
          
        }
      }
    }
  }
}

Stage 3 = 30, LL = -5548.9, AR(0.90) = [0.38], gam = 0.0142, Max-Change = 0.0009

Calculating information matrix...

Calculating log-likelihood...
n = 500 ; j = 10 ; b = -2 ; iter = 1 ; time = 17.28307 
Stage 3 = 34, LL = -7545.3, AR(0.55) = [0.42], gam = 0.0129, Max-Change = 0.0007

Calculating information matrix...

Calculating log-likelihood...
n = 500 ; j = 10 ; b = -2 ; iter = 1 ; time = 25.98505 
Stage 3 = 93, LL = -8355.7, AR(0.46) = [0.42], gam = 0.0060, Max-Change = 0.0009

Calculating information matrix...

Calculating log-likelihood...
n = 500 ; j = 10 ; b = -2 ; iter = 1 ; time = 36.99604 
Stage 3 = 34, LL = -9485.4, AR(0.81) = [0.33], gam = 0.0129, Max-Change = 0.0009

Calculating information matrix...

Calculating log-likelihood...
n = 500 ; j = 20 ; b = -2 ; iter = 1 ; time = 28.1971 
Stage 3 = 55, LL = -12071.7, AR(0.26) = [0.49], gam = 0.0089, Max-Change = 0.0008

Calculating information matrix...

Calculating log-likelihood...
n = 500 ; j = 20 ; b = -2 ; iter = 1 ; ti

ERROR: ignored

In [ ]:
int <- nrow(rdf) / 10
for (i in 1:10) {
  cases <- (int*(i-1)+1):(i*int)
  df <- rdf[cases,]
  write.csv(df, paste0("out", i, ".csv"))
  #print(cases)
}


In [ ]:
for (i in 1:10) {}